In [98]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
from sklearn.preprocessing import KBinsDiscretizer, LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier


train = pd.read_csv('./big-data-analytics-certification/t2-1-train.csv')
test = pd.read_csv('./big-data-analytics-certification/t2-1-test.csv')
sub = pd.read_csv('./big-data-analytics-certification/t2-1-sample_submission.csv')

del train['id']
ids = test['id']
del test['id']
del train['Employment Type']
del test['Employment Type']
# print(train.shape, test.shape)
# print(train.info())
# print()
# print(test.info())
display(train)
# print(train['TravelInsurance'].value_counts(normalize = True))
# class-imbalanced

def kbs(feature, n_bins):
    kb = KBinsDiscretizer(n_bins = n_bins, encode = 'ordinal')
    kb.fit(train[['Age']])
    train[feature] = kb.transform(train[[feature]])
    test[feature] = kb.transform(test[[feature]])

def label_encoder(feature):
    le = LabelEncoder()
    le.fit(train[feature])
    train[feature] = le.transform(train[feature])
    test[feature] = le.transform(test[feature])

train = train.fillna(train['AnnualIncome'].mean())
test = test.fillna(train['AnnualIncome'].mean())
train['Age'] = train['Age']//10
test['Age'] = test['Age']//10
label_encoder('GraduateOrNot')
label_encoder('FrequentFlyer')
label_encoder('EverTravelledAbroad')
sts = StandardScaler()
train['AnnualIncome'] = np.log(train['AnnualIncome'])
test['AnnualIncome'] = np.log(test['AnnualIncome'])


train = pd.get_dummies(train, columns = ['Age'])
test = pd.get_dummies(test, columns = ['Age'])

y = train['TravelInsurance']
del train['TravelInsurance']

x_train, x_test, y_train, y_test = train_test_split(train, y, test_size = 0.2, random_state = 123456, stratify = y)

rf = RandomForestClassifier(n_estimators = 75, random_state = 12346789)
gb = GradientBoostingClassifier(learning_rate = 0.05, n_estimators = 50, max_depth = 5, random_state = 123456789)
xgb = XGBClassifier(n_estimators = 30, learning_rate = 0.07, random_state = 123456789)
rf.fit(x_train, y_train)
gb.fit(x_train, y_train)
xgb.fit(x_train, y_train)
pred = rf.predict_proba(x_test)[:,1]
pred_gb = gb.predict_proba(x_test)[:,1]
pred_xgb = xgb.predict_proba(x_test)[:,1]

#print('acc', accuracy_score(y_test, pred))
print('roc', roc_auc_score(y_test, pred))

#print('acc_gb', accuracy_score(y_test, pred_gb))
print('roc_gb', roc_auc_score(y_test, pred_gb))
print('roc_xgb', roc_auc_score(y_test, pred_xgb))

print(cross_val_score(rf, x_train, y_train, cv = 5, scoring = 'roc_auc').mean())

print(cross_val_score(gb, x_train, y_train, cv = 5, scoring = 'roc_auc').mean())

print(cross_val_score(xgb, x_train, y_train, cv = 5, scoring = 'roc_auc').mean())

# voting = VotingClassifier([('rf',rf),
#                           ('gb', gb),
#                           ('xgb', xgb)], voting = 'hard')

# voting.fit(x_train, y_train)
# pred_voting = voting.predict_proba(x_test)[:,1]
# # print('acc_voting', accuracy_score(y_test, pred_voting))
# print('roc_voting', roc_auc_score(y_test, pred_voting))

# print(cross_val_score(voting, x_train, y_train, cv = 5, scoring = 'roc_auc').mean())


pred = xgb.predict_proba(test)[:,1]
sub['TravelInsurance'] = pred
sub.to_csv('./sub.csv', index = False)

help(XGBClassifier)
# import sklearn as sp
# dir(sm)

,Age,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,28,Yes,1250000.0,6,1,No,No,0
1,31,Yes,1250000.0,7,1,No,No,0
2,29,Yes,1200000.0,7,0,No,No,1
3,33,Yes,650000.0,6,1,No,No,1
4,28,Yes,800000.0,6,0,No,Yes,1
...,...,...,...,...,...,...,...,...
1485,28,Yes,800000.0,4,0,No,No,0
1486,34,Yes,1000000.0,9,0,No,No,0
1487,26,Yes,450000.0,5,1,No,No,1
1488,25,No,1150000.0,3,1,No,Yes,0


roc 0.8119911176905996
roc_gb 0.8006661732050333
roc_xgb 0.8052306933135949
0.7640062740638778
0.7943628184411595
0.8068169349851377
Help on class XGBClassifier in module xgboost.sklearn:

class XGBClassifier(XGBModel, sklearn.base.ClassifierMixin)
 |  XGBClassifier(*, objective: Union[str, Callable[[numpy.ndarray, numpy.ndarray], Tuple[numpy.ndarray, numpy.ndarray]], NoneType] = 'binary:logistic', use_label_encoder: bool = False, **kwargs: Any) -> None
 |  
 |  Implementation of the scikit-learn API for XGBoost classification.
 |  
 |  
 |  Parameters
 |  ----------
 |  
 |      n_estimators : int
 |          Number of boosting rounds.
 |  
 |      max_depth :  Optional[int]
 |          Maximum tree depth for base learners.
 |      max_leaves :
 |          Maximum number of leaves; 0 indicates no limit.
 |      max_bin :
 |          If using histogram-based algorithm, maximum number of bins per feature
 |      grow_policy :
 |          Tree growing policy. 0: favor splitting at nodes 